In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [3]:
%aimport event_selector
%aimport looper_utils
%aimport make_plots

In [4]:
import awkward as ak
import uproot
import numpy as np
from yahist import Hist1D, Hist2D

## Skim Parameters

In [34]:
year = '2018'
skim_version = 'v4'  #currently available x3 or v4

## Load data

In [35]:
%%bash
ls metadata/

chunklist.json
chunklist_2016_v4.json
chunklist_2017_v4.json
chunklist_2018_v4.json
totalWeights.json
totalWeights_2016_v4.json
totalWeights_2017_v4.json
totalWeights_2018_v4.json
xsection.json
xsection_2016_v4.json
xsection_2017_v4.json
xsection_2018_v4.json
xsection_v4.json


In [36]:
import json
with open('./metadata/chunklist_'+year+'_'+skim_version+'.json') as f:
    chunks = json.load(f) 
chunks.keys()

dict_keys(['EGamma_2018A', 'EGamma_2018B', 'EGamma_2018C', 'EGamma_2018D', 'DYJets', 'ttbar', 'ttG', 'ttGG', 'ZG', 'WG', 'GJets_HT40To100', 'GJets_HT100To200', 'GJets_HT200To400', 'GJets_HT400To600', 'GJets_HT600ToInf', 'QCD_pT30To40', 'QCD_pT40ToInf', 'Diphoton', 'VH', 'signal'])

## Process data

In [11]:
def process(args):
    
    import event_selector
    import make_plots
    
    obj_list = ["electron", "muon", "tau", "photon", "others"]
   
    isData    = "EGamma" in args[0] or "DoubleEG" in args[0]
    isSigLike = "HHggtautau" in args[0] or "VH" in args[0]
    events = event_selector.prepare_inputs(args, obj_list, isData=isData)
    
    tot_w = ak.sum(events.genWeight)
    mgg = events.ggMass
    
    ## need to change gHidx
    gHidx = event_selector.get_gHidx(args) 
    g_Hidx = gHidx.gHidx
    #mask_diphoton = event_selector.select_photon_byEvent( events.Photon, g_Hidx, mgg, isSigLike )
    mask_diphoton = event_selector.select_photon_byEvent_v1( events.Photon, mgg, isSigLike )
    
    mask_photon = event_selector.select_photon(events.Photon, g_Hidx, mgg)
    photons_selected = events.Photon[mask_photon] 
    nPho = ak.num(photons_selected)
    
    ## define taus for all categories
    mask_tau_all = event_selector.select_tau(events.Tau, "all", isTight=False)
    mask_tau_all = mask_tau_all & looper_utils.mask_by_dR(events.Tau, photons_selected, 0.2)
    nTau_all = ak.num(events.Tau[mask_tau_all])
    
    mask_tau_tt = event_selector.select_tau(events.Tau, "tau_tau", isTight=True)
    mask_tau_tt = mask_tau_tt & looper_utils.mask_by_dR(events.Tau, photons_selected, 0.2)
    nTau_tt = ak.num(events.Tau[mask_tau_tt])
    
    mask_tau_mt = event_selector.select_tau(events.Tau, "tau_mu", isTight=True)
    mask_tau_mt = mask_tau_mt & looper_utils.mask_by_dR(events.Tau, photons_selected, 0.2)
    nTau_mt = ak.num(events.Tau[mask_tau_mt])
    
    mask_tau_et = event_selector.select_tau(events.Tau, "tau_e", isTight=True)
    mask_tau_et = mask_tau_et & looper_utils.mask_by_dR(events.Tau, photons_selected, 0.2)
    nTau_et = ak.num(events.Tau[mask_tau_et])
    
    mask_tau_lt = mask_tau_mt | mask_tau_et
    
    ## loose leptons
    mask_eleL = event_selector.select_electron(events.Electron, isTight=False)
    mask_eleL = mask_eleL & looper_utils.mask_by_dR(events.Electron, photons_selected, 0.2)
    nEleL = ak.num(events.Electron[mask_eleL])
    
    mask_muL = event_selector.select_muon(events.Muon, isTight=False)
    mask_muL = mask_muL & looper_utils.mask_by_dR(events.Muon, photons_selected, 0.2)
    nMuL = ak.num(events.Muon[mask_muL])

    ## tight leptons
    mask_eleT = event_selector.select_electron(events.Electron, isTight=True)
    mask_eleT = mask_eleT & looper_utils.mask_by_dR(events.Electron, photons_selected, 0.2)
    nEleT = ak.num(events.Electron[mask_eleT])
    
    mask_muT = event_selector.select_muon(events.Muon, isTight=True)
    mask_muT = mask_muT & looper_utils.mask_by_dR(events.Muon, photons_selected, 0.2)
    nMuT = ak.num(events.Muon[mask_muT])
    
    sum_charge_tt = (ak.sum(events.Tau[mask_tau_tt].charge, axis=1) == 0)
    sum_charge_et = (ak.sum(events.Tau[mask_tau_et].charge, axis=1)    + ak.sum(events.Electron[mask_eleT].charge, axis=1) == 0)
    sum_charge_mt = (ak.sum(events.Tau[mask_tau_mt].charge, axis=1)    + ak.sum(events.Muon[mask_muT].charge, axis=1)      == 0)
    sum_charge_ll = (ak.sum(events.Electron[mask_eleT].charge, axis=1) + ak.sum(events.Muon[mask_muT].charge, axis=1)      == 0)
    mask_dR_mll_0lep_2tau = looper_utils.mask_by_dR_mll_0lep_2tau(events.Tau[mask_tau_tt], (0.2,3.5), (30,140))
    mask_dR_mll_1mu_1tau  = looper_utils.mask_by_dR_mll_1lep_1tau(events.Tau[mask_tau_mt], events.Muon[mask_muT]     , (0.3,3.5), (20,120))
    mask_dR_mll_1ele_1tau = looper_utils.mask_by_dR_mll_1lep_1tau(events.Tau[mask_tau_et], events.Electron[mask_eleT], (0.3,3.5), (20,120))
    
    mask_dipho     = mask_diphoton &  (nPho == 2) 
    mask_0lep_1tau = mask_dipho & (nTau_all == 1) & (nEleL == 0) & (nMuL == 0)
    mask_0lep_2tau = mask_dipho & (nTau_tt  == 2) & (nEleL == 0) & (nMuL == 0) & (sum_charge_tt) & (mask_dR_mll_0lep_2tau)
    mask_2lep_0tau = mask_dipho & (nTau_all == 0) & (nEleT + nMuT == 2) & (sum_charge_ll)
    mask_1lep_0tau = mask_dipho & (nTau_all == 0) & (nEleT + nMuT == 1)
    
    mask_1mu_1tau  = mask_dipho & (nTau_mt == 1) & (nEleL + nMuL == 1) & (nMuT  == 1) & (sum_charge_mt) & (mask_dR_mll_1mu_1tau)
    mask_1ele_1tau = mask_dipho & (nTau_et == 1) & (nEleL + nMuL == 1) & (nEleT == 1) & (sum_charge_et) & (mask_dR_mll_1ele_1tau)
    mask_1lep_1tau = (mask_1mu_1tau) | (mask_1ele_1tau)
    
    masks = {"dipho": mask_dipho,
             "0lep_1tau": mask_0lep_1tau,
             "0lep_2tau": mask_0lep_2tau,
             "1lep_1tau": mask_1lep_1tau,
             "2lep_0tau": mask_2lep_0tau,
             "1lep_0tau": mask_1lep_0tau}
    
    hists = {}
    
    for key, mask in masks.items():
        hists[key] = make_plots.process_event( events[mask], events.genWeight[mask] )
        hists[key].update( make_plots.process_diphoton(events.Photon[mask], g_Hidx[mask], mgg[mask], events.genWeight[mask]) )
        if key == 'dipho':
            hists[key].update( make_plots.process_tau(     events.Tau[mask_tau_all][mask]  , events.genWeight[mask]))
            hists[key].update( make_plots.process_muon(    events.Muon[mask_muL][mask]     , events.genWeight[mask]))
            hists[key].update( make_plots.process_electron(events.Electron[mask_eleL][mask], events.genWeight[mask]))
        if key == '0lep_1tau':
            hists[key].update( make_plots.process_tau(     events.Tau[mask_tau_all][mask]  , events.genWeight[mask]))
        if key == '0lep_2tau':
            hists[key].update( make_plots.process_tau(     events.Tau[mask_tau_tt][mask]   , events.genWeight[mask]))
        if key == '1lep_0tau' or key == '2lep_0tau':
            hists[key].update( make_plots.process_muon(    events.Muon[mask_muT][mask]     , events.genWeight[mask]))
            hists[key].update( make_plots.process_electron(events.Electron[mask_eleT][mask], events.genWeight[mask]))
        if key == "1lep_1tau":
            tmphists = hists[key].update( make_plots.process_1tau_1lep(events.Tau[mask_tau_lt][mask], events.Muon[mask_muT][mask],
                                                                       events.Electron[mask_eleT][mask], events.genWeight[mask]))
    
    return hists

In [12]:
%%time
hists = process(chunks["signal"][0])
#hists = process((chunks["signal"][0][0], 0, 200) )
#hists = process(chunks["ZH"][0])

CPU times: user 9.55 s, sys: 229 ms, total: 9.78 s
Wall time: 11 s


In [13]:
#hists['1lep_1tau']['tautauL_SVFit']
hists['1lep_1tau']['MET']

[ 5.00 ± 2.24   13.00 ± 3.61   17.00 ± 4.12   17.00 ± 4.12   28.00 ± 5.29
 29.00 ± 5.39   39.98 ± 6.32   34.00 ± 5.83   25.00 ± 5.00   28.00 ± 5.29
 46.00 ± 6.78   46.00 ± 6.78   43.00 ± 6.56   45.00 ± 6.71   35.00 ± 5.92
 38.00 ± 6.16   22.00 ± 4.69   32.99 ± 5.74   31.00 ± 5.57   28.00 ± 5.29
 34.00 ± 5.83   18.99 ± 4.36   19.00 ± 4.36   20.00 ± 4.47   27.99 ± 5.29
 14.99 ± 3.87   21.00 ± 4.58   22.00 ± 4.69   18.00 ± 4.24   14.00 ± 3.74
 16.00 ± 4.00   12.00 ± 3.46    9.00 ± 3.00   15.00 ± 3.87   14.00 ± 3.74
  9.00 ± 3.00   15.00 ± 3.87    7.00 ± 2.65    4.99 ± 2.23   11.00 ± 3.32
  6.00 ± 2.45    5.00 ± 2.24    5.00 ± 2.24    4.00 ± 2.00    3.00 ± 1.73
  4.00 ± 2.00    4.00 ± 2.00    7.00 ± 2.65    3.00 ± 1.73    5.00 ± 2.24
  6.00 ± 2.45    5.00 ± 2.24    2.00 ± 1.41    2.00 ± 1.41    2.00 ± 1.41
  1.00 ± 1.00    3.00 ± 1.73    3.00 ± 1.73    2.00 ± 1.41    2.00 ± 1.41
  2.00 ± 1.41    1.00 ± 1.00    2.00 ± 1.41    1.00 ± 1.00    1.00 ± 1.00
  1.00 ± 1.00    1.00 ± 1.00    2.00 ± 1.41    3.00 ± 1.73    1.00 ± 1.00
  2.00 ± 1.41    1.00 ± 1.00    1.00 ± 1.00    2.00 ± 1.41    1.00 ± 1.00
  0.00 ± 0.00    2.00 ± 1.41    2.00 ± 1.41    2.00 ± 1.41    1.00 ± 1.00
  2.00 ± 1.41    0.00 ± 0.00    0.00 ± 0.00    2.00 ± 1.41    1.00 ± 1.00
  0.00 ± 0.00    2.00 ± 1.41    0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00
  0.00 ± 0.00    1.00 ± 1.00    0.00 ± 0.00    0.00 ± 0.00    0.00 ± 0.00
  0.00 ± 0.00    2.00 ± 1.41    1.00 ± 1.00    0.00 ± 0.00    3.00 ± 1.73]

In [14]:
hists['dipho']['pho_pT1'].integral
#hists['dipho']['pho_pT1'].integral_error

17863.510528564453

## send to dask

In [1]:
#from dask.distributed import Client
#client = Client('tcp://169.228.130.74:5608')
#client

Client Scheduler: tcp://169.228.130.74:5608 Dashboard: http://169.228.130.74:12332/status,Cluster Workers: 15 Cores: 15 Memory: 60.00 GB


In [14]:
from dask.distributed import Client
client = Client(memory_limit='4GB', n_workers=12, threads_per_worker=1)
client

Client Scheduler: tcp://127.0.0.1:19598 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 48.00 GB


In [37]:
def g():
    import glob
    allpys = glob.glob("./*py")
    return allpys
g()

['./__init__.py',
 './cachepreload.py',
 './condor_utils.py',
 './event_selector.py',
 './loop.py',
 './looper_utils.py',
 './make_plots.py',
 './utils.py',
 './make_plots_from_json.py']

In [38]:
x = client.submit(g)

In [39]:
client.gather(x)

['./__init__.py',
 './cachepreload.py',
 './condor_utils.py',
 './event_selector.py',
 './loop.py',
 './looper_utils.py',
 './make_plots.py',
 './utils.py',
 './make_plots_from_json.py']

In [40]:
%%time
from dask.distributed import as_completed
import collections, functools, operator 

futures = {}
results = {}

cat_keys = ["dipho", "0lep_1tau", "0lep_2tau", "1lep_1tau", "2lep_0tau", "1lep_0tau"]

for process_key in chunks:
    print ("start process {}".format(process_key))
    futures[process_key] = client.map(process, chunks[process_key], retries=5)
    
    results_local = []
    results[process_key] = {}
    ## do you really need this while?
    while len(results_local) < len(chunks[process_key]): 
        ac = as_completed(futures[process_key], with_results=True)
        for future, result in ac:
            results_local.append(result)
    
    ## merge histograms by the key
    for cat_key in cat_keys: 
        dicts = [results_local[i][cat_key] for i in range(len(results_local))]  
        counter = collections.Counter() 
        for d in dicts:  
            counter.update(d) 
        results[process_key][cat_key] = dict(counter) 

start process EGamma_2018A
start process EGamma_2018B
start process EGamma_2018C
start process EGamma_2018D
start process DYJets
start process ttbar
start process ttG
start process ttGG
start process ZG
start process WG
start process GJets_HT40To100
start process GJets_HT100To200
start process GJets_HT200To400
start process GJets_HT400To600
start process GJets_HT600ToInf
start process QCD_pT30To40
start process QCD_pT40ToInf
start process Diphoton
start process VH
start process signal
CPU times: user 2min 47s, sys: 24.7 s, total: 3min 11s
Wall time: 17min 25s


## save to disk?

- foler1: cat (dipho, 0lep_1tau, 0lep_2tau...)
    + folder2: process (data, signal, ZG...)
        - folder3: hists (pT, eta, phi...)

In [ ]:
client.shutdown()

In [41]:
results.keys()

dict_keys(['EGamma_2018A', 'EGamma_2018B', 'EGamma_2018C', 'EGamma_2018D', 'DYJets', 'ttbar', 'ttG', 'ttGG', 'ZG', 'WG', 'GJets_HT40To100', 'GJets_HT100To200', 'GJets_HT200To400', 'GJets_HT400To600', 'GJets_HT600ToInf', 'QCD_pT30To40', 'QCD_pT40ToInf', 'Diphoton', 'VH', 'signal'])

In [42]:
results['signal'].keys()

dict_keys(['dipho', '0lep_1tau', '0lep_2tau', '1lep_1tau', '2lep_0tau', '1lep_0tau'])

In [43]:
results['signal']['dipho'].keys()

dict_keys(['MET', 'tautau_SVFit', 'tautauA_SVFit', 'tautauL_SVFit', 'pho_pT1', 'pho_pT2', 'pho_pTom1', 'pho_pTom2', 'pho_eta1', 'pho_eta2', 'pho_phi1', 'pho_phi2', 'pho_id1', 'pho_id2', 'tau_pT1', 'tau_eta1', 'tau_phi1', 'tau_deeptau_vs_j_1', 'tau_deeptau_vs_m_1', 'tau_deeptau_vs_e_1', 'n_tau', 'mtautau', 'dR_tautau', 'tau_pT2', 'tau_eta2', 'tau_phi2', 'tau_deeptau_vs_j_2', 'tau_deeptau_vs_m_2', 'tau_deeptau_vs_e_2', 'muon_pT1', 'muon_eta1', 'muon_phi1', 'muon_iso1', 'n_muon', 'mmumu', 'dR_mumu', 'muon_pT2', 'muon_eta2', 'muon_phi2', 'muon_iso2', 'electron_pT1', 'electron_eta1', 'electron_phi1', 'electron_iso1', 'n_electron', 'mee', 'dR_ee', 'electron_pT2', 'electron_eta2', 'electron_phi2', 'electron_iso2'])

In [44]:
%%time
from subprocess import call
import json

process_keys = results.keys()
cat_keys = results['signal'].keys()

tag = "tight_dR_mll_cut_blind"

for cat_key in cat_keys:
    for process_key in process_keys:
        dirname = './hists/' + tag + '/' + cat_key + '/' + process_key + '/'
        call('mkdir -p ' + dirname, shell=True)
        for hist_key in results[process_key][cat_key].keys():
            histname = dirname + hist_key +'_'+year+'_'+skim_version
            results[process_key][cat_key][hist_key].to_json(histname+".json")
            with open(histname, "w") as f:
                data = json.dump(results[process_key][cat_key][hist_key].to_json(histname+".json"), f)

CPU times: user 2.19 s, sys: 3.85 s, total: 6.04 s
Wall time: 13.9 s
